In [4]:
import pathlib
from pathlib import Path
from typing import List, Tuple, Union
import copy
import numpy as np
import matplotlib.pyplot as plt
import serpentTools
from serpentTools.parsers.results import ResultsReader
import click
import matplotlib.pyplot as plt

serpentTools.settings.rc["serpentVersion"] = "2.1.32"


AV = 0,6221408E+24


def discharge_bu() -> None:
    res_list = []

    with serpentTools.settings.rc:
        serpentTools.settings.rc["verbosity"] = "error"
    dep = serpentTools.read("wh_lfr_dep.m")
    
    return dep

if __name__ == "__main__":
    dep = discharge_bu()


Executing <Task pending coro=<Kernel.dispatch_queue() running at /home/stalex/repos/01_wh_lfr/.direnv/python-3.7.13/lib/python3.7/site-packages/ipykernel/kernelbase.py:510> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7f5437431540>()] created at /home/stalex/repos/01_wh_lfr/.direnv/python-3.7.13/lib/python3.7/site-packages/tornado/queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/stalex/repos/01_wh_lfr/.direnv/python-3.7.13/lib/python3.7/site-packages/tornado/ioloop.py:687] created at /home/stalex/repos/01_wh_lfr/.direnv/python-3.7.13/lib/python3.7/site-packages/tornado/gen.py:874> took 1.124 seconds


In [ ]:
#print(dep.materials["fuelP1Z1"].mdens[:,3][:38])
#print(len(dep.materials["fuelP1Z1"].mdens[:,3][:38]))
print(dep.names[:39])
m_dens = dep.materials["fuelP1Z1"].mdens[:,3][:39]
names = np.array(dep.names[:39])
print(names)
print(m_dens)
print(np.vstack((names,m_dens)))




In [19]:
P = 48 #max no of FA
Z = 6  #max no of slices

last_bu_step = len(dep.burnup) - 1
max_list_HM = np.where(np.char.find(dep.names, "Cm250") == 0)[0][0] + 1

# Gets the correction ration for FA
c_ratio = []
name_list = []
serpent_bu = []
corrected_bu = []

step = 0

for poz in range(1, P+1):
    for slice in range(1, Z+1):
        fa_name = f"fuelP{poz}Z{slice}"
        fa1_name = f"fuelP1Z{slice}"

        sum_hm = np.sum(dep.materials[fa_name].mdens[: ,last_bu_step][:max_list_HM])
        sum_hm_fa1 = np.sum(dep.materials[fa1_name].mdens[: ,last_bu_step][:max_list_HM])
        ratio_i = sum_hm/sum_hm_fa1

        c_ratio.append(ratio_i)
        name_list.append(fa_name)

        
        serpent_bu.append(dep.materials[fa_name].burnup[last_bu_step])

        #print(step)
        if step <= 5:
            #print("FA1")
            corrected_bu.append(dep.materials[fa1_name].burnup[last_bu_step])
        else:    
            #print("FAn")
            fa_c_bu = dep.materials[fa_name].burnup[last_bu_step] * c_ratio[step-6]
            #print(c_ratio[step-6])
            corrected_bu.append(fa_c_bu)

        

        step += 1

#np.vstack((name_list,c_ratio))
#print(len(corrected_bu))
#print(corrected_bu)



In [21]:
type(serpent_bu)


list